In [ ]:
    using Pkg
    Pkg.activate("..")
    using Test: Random
    using JLD2
    using Printf, Statistics
    using Profile
    using MYMLP, Printf, Statistics

In [ ]:
    X_train = load("../data/KM3/imdb_dataset_prepared.jld2", "X_train")
    y_train = load("../data/KM3/imdb_dataset_prepared.jld2", "y_train")
    X_test = load("../data/KM3/imdb_dataset_prepared.jld2", "X_test")
    y_test = load("../data/KM3/imdb_dataset_prepared.jld2", "y_test")
    embeddings = load("../data/KM3/imdb_dataset_prepared.jld2", "embeddings")
    vocab = load("../data/KM3/imdb_dataset_prepared.jld2", "vocab")
    dataset = DataLoader((X_train, y_train), batchsize=64, shuffle=true)
    embedding_dim = size(embeddings, 1)

    fixed_randn32(dims...) = randn32(Random.MersenneTwister(123), dims...)
    fixed_glorot(dims...) = glorot_uniform(Random.MersenneTwister(123), dims...)
    
    my_model = Chain(
        Embedding(length(vocab), embedding_dim, init=fixed_randn32),
        x -> permutedims(x, (2, 1, 3)),
        Conv((3,), embedding_dim => 8, relu, init=fixed_glorot),
        MaxPool((8,)),
        flatten,
        Dense(128, 1, sigmoid, init=fixed_glorot)
    )